In [ ]:
#Importing Libraries
from keras.layers import Input,Dense,Flatten
from keras.models import Model
from tensorflow.keras import layers
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [ ]:
train_path = '../input/100-bird-species/train'
valid_path = '../input/100-bird-species/valid'
test_path = '../input/100-bird-species/test'

In [ ]:
# add preprocessing layer to the front of EfficientNet
xcept = Xception(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
# don't train existing weights
for layer in xcept.layers:
  layer.trainable = False

In [ ]:
#Getting Number of Categories
folders = glob('../input/100-bird-species/train/*')

In [ ]:
x = Flatten()(xcept.output)

In [ ]:
x = layers.Dense(256, 'relu', kernel_initializer='he_normal')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
# create a model object
model = Model(inputs=xcept.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
#Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

valid_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('../input/100-bird-species/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
valid_set = valid_datagen.flow_from_directory('../input/100-bird-species/valid',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('../input/100-bird-species/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
#Fitting the model
r = model.fit(
  training_set,
  validation_data=valid_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(valid_set)
)

In [ ]:
#Saving the Model.
import tensorflow as tf

from keras.models import load_model

model.save('./bird_classification_new_model.h5')

In [ ]:
model.evaluate(test_set)